In [51]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget
import h5py
import hdf5plugin

import deconvolution
import utils
import xtrace

import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

#Data Files

project_dir = '/data/visitors/balder/20220115/2022022408'
experiment_dir = os.path.join(project_dir, 'raw/XRD Eiger Test')
poni_dir = os.path.join(project_dir, 'process')

stacks = {
    "no_tilt_close": {
        "filename": 'LaB6_9_data_000001.h5',
        "poni": 'samuel_9_0.poni'
    },
    "large_tilt_close": {
       "filename": 'LaB6_10_data_000001.h5',
       "poni": 'samuel_10_0.poni'
    },
    #"small_tilt_close": {
    #    "filename": 'LaB6_11_data_000001.h5',
    #    "poni": 'samuel_11_0.poni'
    #},
    #"no_tilt_far": {
    #    "filename": 'LaB6_13_data_000001.h5',
    #    "poni": 'samuel_13_0.poni'
    #}
}

icsdfilepath = os.path.join(experiment_dir,'LaB6-icsd.txt')

az_npt = 2000
data_layer = 0
radial = (0, 60)

stack = stacks["no_tilt_close"]
config, data = utils.load_stack(
    os.path.join(project_dir, experiment_dir, stack["filename"]),
    os.path.join(poni_dir, stack["poni"]),
)
img, mask = correction.mask(data[data_layer], False)
integrator, det_params_b711, tth_hkl = utils.azimutal_fit(config, icsdfilepath)

def deconvolve(config, img, samp_density):
    ray_grid = utils.ray_grid(config["dimensions"], samp_density)
    G = xtrace.depth_spill_psf(config, *ray_grid)
    recovered_img = deconvolution.richard_lucy(img, G, 10)
    return recovered_img, G


recovered_img_single, G_single = deconvolve(config, img, 1)
recovered_img_mult, G_mult = deconvolve(config, img, 5)
config_upsamp, img_upsamp = utils.upsample_transform(config, img)
recovered_img_upsamp_highres, G_upsamp = deconvolve(config_upsamp, img_upsamp, 1)
recovered_img_upsamp = utils.downsample_img(recovered_img_upsamp_highres)

grid_img = np.zeros(img.shape)
grid_img[::10,::10] = 1;
recovered_grid_single = (G_single.get()@grid_img.flatten()).reshape(grid_img.shape)
recovered_grid_mult = (G_mult.get()@grid_img.flatten()).reshape(grid_img.shape)
grid_img_highres = np.zeros((img.shape[0]*2, 2*img.shape[1]))
grid_img_highres[::20,::20] = 1; grid_img_highres[1::20,::20] = 1
grid_img_highres[::20,1::20] = 1; grid_img_highres[1::20,1::20] = 1
recovered_grid_upsamp = (G_upsamp.get()@grid_img_highres.flatten()).reshape(grid_img_highres.shape)

integrated_img = integrator.integrate2d(img, az_npt, az_npt, radial_range=radial)[0]

perc = np.percentile(img, 99.6)

fig, axs = plt.subplots(2, 4, figsize=(10, 5), sharex=True, sharey=True)

axs[0,0].imshow(img, vmax=perc); axs[0,0].set_title("Original")
axs[0,1].imshow(recovered_img_single, vmax=perc); axs[0,1].set_title("1x1 RPP")
axs[0,2].imshow(recovered_img_mult, vmax=perc); axs[0,2].set_title("5x5 RPP")
axs[0,3].imshow(recovered_img_upsamp, vmax=perc); axs[0,3].set_title("1x1 RPP, UPSAMP 2x")

axs[1,0].imshow(grid_img)
axs[1,1].imshow(recovered_grid_single)
axs[1,2].imshow(recovered_grid_mult)
axs[1,3].imshow(recovered_grid_upsamp, extent=[-0.5,img.shape[1]-0.5,img.shape[0]-0.5,-0.5])

# plt.figure()
# ax = plt.axes(projection='3d')
# part = img[590:610,155:175]
# x, y = np.meshgrid(np.arange(part.shape[1]), np.arange(part.shape[0]))
# ax.plot_surface(x, y, part, cmap='viridis', edgecolor='none')
# ax.set_title('Original')
# plt.figure()
# plt.imshow(part)
# plt.figure()
# ax = plt.axes(projection='3d')
# part = recovered_img_mult[590:610,155:175]
# x, y = np.meshgrid(np.arange(part.shape[1]), np.arange(part.shape[0]))
# ax.plot_surface(x, y, part, cmap='viridis', edgecolor='none')
# ax.set_title('recovered 5x5 RPP')
# plt.figure()
# plt.imshow(part)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …